<a href="https://colab.research.google.com/github/CafeAAndrade/Tutor-IA-Personalizado---Projeto-Alura/blob/main/Tutor%20IA%20Personalizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai
%pip -q install google-adk

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, Image, HTML # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings


warnings.filterwarnings("ignore")

In [10]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str):
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"

    return final_response

In [11]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
OPEN_QUESTIONS = [
    {
        "id": "q1",
        "text": "1. Quando você precisa aprender algo completamente novo e um pouco complexo, como você geralmente prefere começar? Descreva os primeiros passos que você daria."
    },
    {
        "id": "q2",
        "text": "2. Pense em uma aula ou palestra que você achou particularmente eficaz. O que o instrutor fez ou quais materiais foram usados que te ajudaram a aprender?"
    },
    {
        "id": "q3",
        "text": "3. Se você tivesse que explicar um conceito difícil para um amigo, como você faria isso? Quais recursos ou métodos você usaria?"
    },
    {
        "id": "q4",
        "text": "4. Ao se preparar para uma prova ou apresentação importante, quais são suas técnicas de estudo ou preparação preferidas? Descreva como você as utiliza."
    },
    {
        "id": "q5",
        "text": "5. Quando você está tentando entender instruções para montar algo (como um móvel) ou seguir uma receita, que tipo de informação ou formato te ajuda mais?"
    },
    {
        "id": "q6",
        "text": "6. Descreva um hobby ou atividade de lazer que você gosta e explique como você aprendeu ou como você se aprimora nele."
    },
    {
        "id": "q7",
        "text": "7. Se você estivesse pesquisando um novo gadget ou produto antes de comprar, que tipo de informação você buscaria e onde?"
    },
    {
        "id": "q8",
        "text": "8. O que mais te distrai quando você está tentando se concentrar para aprender algo novo?"
    },
    {
        "id": "q9",
        "text": "9. Para memorizar informações, como números de telefone, datas ou listas, quais estratégias você acha que funcionam melhor para você?"
    },
    {
        "id": "q10",
        "text": "10. Se você pudesse projetar a 'aula ideal' para você, como ela seria? Descreva o ambiente, os métodos de ensino e os materiais."
    }
]

In [30]:
def run_interview_open(questions):
    """
    Conduz a entrevista com o usuário, fazendo perguntas abertas e coletando respostas.
    Retorna um dicionário com as respostas do usuário.
    """

    print("Agora vou fazer algumas perguntas abertas. Por favor, descreva suas preferências e experiências com o máximo de detalhes que puder.\n")

    answers = {}
    for q_data in questions:
        print(f"{q_data['text']}")
        answer = input("Sua resposta: ").strip()
        answers[q_data['id']] = answer
        print("-" * 30)

    return answers

In [31]:
def generate_prompt_for_gemini(user_answers, questions_list):
    """
    Gera o prompt que será enviado ao modelo Gemini.
    """
    prompt = "\nRespostas do Usuário:\n"
    for q_data in questions_list:
        q_id = q_data['id']
        q_text = q_data['text']
        if q_id in user_answers:
            prompt += f"\nPergunta {q_data['id'][-1]}: {q_text}\nResposta do Usuário: {user_answers[q_id]}\n" # Usando apenas o número da pergunta

    prompt += "\nPor favor, gere o relatório sucinto e objetivo, bem estruturado em Markdown, como se estivesse falando diretamente com o usuário."
    return prompt

In [26]:
##########################################
# --- Agente 1: Avaliador --- #
##########################################
def agente_avaliador(prompt):

  avaliador=Agent(
      name="agente_avaliador",
      model= MODEL_ID,
      description="Agente que avalia os resultados das perguntas de perfil VARK",
      tools=[google_search],
      instruction="""
        Você é um especialista em estilos de aprendizagem, com foco no modelo VARK (Visual, Auditivo, Leitura/Escrita, Cinestésico).\n
        Analise as seguintes respostas fornecidas por um usuário a um questionário sobre suas preferências de aprendizagem.\n
        Para cada resposta, identifique indicadores dos quatro estilos VARK.\n
        Ao final, determine o(s) estilo(s) de aprendizagem predominante(s) do usuário.\n
        Gere um relatório detalhado e personalizado para o usuário, em Português do Brasil, que inclua:\n
        1. Uma breve e amigável introdução sobre o que são estilos de aprendizagem VARK e a importância de conhecê-los.\n
        2. O(s) estilo(s) de aprendizagem que parecem ser predominantes para o usuário, com uma explicação clara do porquê essa conclusão foi tirada,
        citando exemplos das respostas do usuário (sem revelar as respostas inteiras, mas referenciando as ideias principais).\n
        3. Para CADA UM dos estilos VARK predominantes do usuário (Visual, Auditivo, Leitura/Escrita, Cinestésico), forneça:\n
            a. Uma descrição detalhada do que caracteriza um aprendiz com esse estilo.\n
            b. Recomendações específicas e práticas de estudo, técnicas de aprendizado e tipos de conteúdo que são mais adequados para maximizar o aprendizado de alguém com esse estilo.\n
        4. Se mais de um estilo parecer predominante, explique como eles podem se complementar e como o usuário pode usar essa combinação a seu favor.\n
        5. Uma conclusão encorajadora, lembrando que os estilos são preferências e que a flexibilidade é importante.\n\n
      """
  )

  entrada_do_agente_avaliador = f" {prompt} "

  perfil = call_agent(avaliador, entrada_do_agente_avaliador)

  return perfil

In [34]:
##########################################
# --- Agente 2: Pesquisador --- #
##########################################
def agente_pesquisador(topico):

  pesquisador=Agent(
      name="agente_pesquisador",
      model= MODEL_ID,
      description="Agente que pesquisa profundamente o topico escolhido",
      tools=[google_search],
      instruction="""
      Você é um pesquisador experiente. Entende de muitos assuntos e tem uma paixão quase obsessiva por encontrar os motivos e explicações para as coisas.
      Você é incansável em buscar mais informações usando a ferramenta google search (google_search) para buscar diversas fontes de informação.
      Para você uma pesquisa só está completa após verificar e cruzar múltiplas fontes de informação.
      Seus relatórios são profundos e detalhados, normalmente superando 5000 palavras.
      Você preza muito a qualidade das informações, sempre se baseando apenas nos fatos.

      """
  )

  entrada_do_agente_pesquisador = f"Pesquise extensivamente e prepare um relatório maravilhoso, detalhado, objetivo e fact based para: {topico} "

  pesquisa = call_agent(pesquisador, entrada_do_agente_pesquisador)

  return pesquisa

In [36]:
##########################################
# --- Agente 3: Professor --- #
##########################################
def agente_professor(resultado_pesquisa, nome, idade, profissao):

  professor=Agent(
      name="agente_professor",
      model= MODEL_ID,
      description="Agente que adequa a explicação do tópico ao perfil etário, profissional e de aprendizado",
      tools=[google_search],
      instruction="""
      Você é um professor inspirador sem igual. É apaixonado por ensinar e busca transmitir sua paixão pelos assuntos para os seus alunos.
      Você trabalha junto com o pesquisador, que te entrega um relatorio completo, mas muito seco, que precisa obrigatoriamente ser ajustado
      para refletir o seu estilo, objetivo, apaixonado por ensinar e cuidadoso para ajudar o aluno a aprender melhor.
      Você sempre adapta o seu conteúdo e muitas vezes até complementa o material do pesquisador usando a ferramenta google search (google_search)
      para trazer exemplos e paralelos que possam ajudar nas explicações.
      Você faz todo o possível para que seus alunos tenham sucesso no aprendizado

      """
  )

  entrada_do_agente_professor = f"Seu aluno se chama {nome}, tem {idade} anos e tem a profissão de {profissao} \n Prepare uma aula memorável para ele sobre: {topico} "

  aula = call_agent(professor, entrada_do_agente_professor)

  return aula

In [38]:
print("Olá! Sou seu agente de IA para te ajudar a aprender alavancando seu estilo natural de aprendizagem. \n Primeiro, me ajude com as informações abaixo:\n")

nome = input("Qual o seu nome")
idade = input("Qual a sua idade?")
profissao = input("Qual a sua profissão?")
topico = input("Qual o topico que você quer estudar?")

    # 1. Coletar respostas abertas
user_answers_collected = run_interview_open(OPEN_QUESTIONS)

    # 2. Gerar o prompt para o Gemini
gemini_prompt = generate_prompt_for_gemini(user_answers_collected, OPEN_QUESTIONS)

perfil_usuario = agente_avaliador(gemini_prompt)
print("\n Resultados do Agente Buscador:\n")
display(to_markdown(perfil_usuario))
print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")

print(f"Pesquisando sobre: {topico} ...")

pesquisa_concluida = agente_pesquisador(topico)
#print("\n Resultados do Agente Buscador:\n")
#display(to_markdown(pesquisa_concluida))
#print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")

print(f"Preparando sua aula sobre: {topico} ...")

aula_concluida = agente_professor(pesquisa_concluida, nome, idade, profissao)
print("\n Resultados do Agente Professor:\n")
display(to_markdown(aula_concluida))
print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\n")

print("\nEspero que este relatório detalhado ajude você a otimizar seus estudos e aprendizado! 😊")

Olá! Sou seu agente de IA para te ajudar a aprender alavancando seu estilo natural de aprendizagem. 
 Primeiro, me ajude com as informações abaixo:

Qual o seu nomeCafe
Qual a sua idade?10
Qual a sua profissão?estudante
Qual o topico que você quer estudar?ia
Agora vou fazer algumas perguntas abertas. Por favor, descreva suas preferências e experiências com o máximo de detalhes que puder.

1. Quando você precisa aprender algo completamente novo e um pouco complexo, como você geralmente prefere começar? Descreva os primeiros passos que você daria.
Sua resposta: leitura
------------------------------

 Resultados do Agente Buscador:



> Olá! Vamos descobrir juntos como você aprende melhor?
> 
> Identificar seu estilo de aprendizagem VARK (Visual, Auditivo, Leitura/Escrita, Cinestésico) pode te ajudar a otimizar seus estudos e a absorver informações de forma mais eficaz. Não existe um estilo "certo" ou "errado", e a maioria das pessoas se beneficia de uma combinação deles. O importante é entender suas preferências para aproveitar ao máximo seu potencial!
> 
> **Seu(s) Estilo(s) Predominante(s)**
> 
> Baseado na sua resposta à primeira pergunta, a leitura parece ser um estilo de aprendizagem importante para você. Ao indicar que sua primeira abordagem para aprender algo novo é a leitura, você demonstra uma preferência por informações apresentadas em formato textual.
> 
> **Estilo de Aprendizagem Leitura/Escrita**
> 
> *   **O que te define:** Pessoas com preferência por Leitura/Escrita aprendem melhor através de palavras escritas. Livros, artigos, textos, anotações e listas são seus melhores amigos. Você provavelmente gosta de fazer anotações detalhadas e se sente mais confortável quando as informações são apresentadas de forma organizada e textual.
> *   **Como turbinar seus estudos:**
>     *   **Anotações:** Transforme aulas e palestras em anotações bem estruturadas. Use diferentes cores e canetas para destacar pontos importantes.
>     *   **Resumos:** Faça resumos dos seus materiais de estudo. Escrever ajuda a fixar o conteúdo.
>     *   **Listas:** Crie listas de tarefas, conceitos-chave e informações relevantes. A organização visual te ajudará a memorizar.
>     *   **Materiais de leitura:** Priorize livros, artigos e apostilas bem escritos e detalhados. Evite materiais excessivamente visuais ou informais.
>     *   **Reescreva:** Reescreva suas anotações e resumos com suas próprias palavras. Isso reforça o aprendizado e te ajuda a entender o conteúdo em profundidade.
> 
> **Conclusão**
> 
> Lembre-se que o VARK é um guia, não uma sentença! Explore diferentes estilos e descubra o que funciona melhor para você em cada situação. A flexibilidade é a chave para se tornar um aprendiz eficaz e completo.
> 
> 


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

pesquisando sobre: ia
Preparando sua aula sobre: ia

 Resultados do Agente Professor:



> Olá, Cafe! Que legal ter você aqui para mais uma aula superinteressante! 😄 Hoje, vamos explorar um mundo mágico chamado Inteligência Artificial, ou IA! 🤖✨
> 
> Sabe, Cafe, a IA é como ter um amigo muito esperto que mora dentro dos computadores e máquinas, e que está sempre aprendendo coisas novas! Ele pode te ajudar em muitas tarefas e até criar coisas incríveis! 🤩
> 
> Para começarmos nossa aventura, vou te contar um segredo: a IA já faz parte do seu dia a dia! Quer ver só?
> 
> 
> *   Quando você pede para a Alexa tocar sua música favorita, é a IA que entende o que você falou e escolhe a música certa.
> *   Sabe os jogos no celular ou tablet que parecem "aprender" como você joga? É a IA tentando tornar o jogo mais divertido e desafiador para você.
> *   E quando você usa um aplicativo que transforma seu rosto em um personagem engraçado? Isso também é IA em ação! 😄
> 
> A IA está presente em diversas ferramentas e plataformas que personalizam o ensino de acordo com o ritmo e as dificuldades de cada criança.
> 
> **Mas, como a IA faz tudo isso?** 🤔
> 
> Imagine que a IA é como um detetive que observa muitos e muitos exemplos para aprender. Por exemplo, para reconhecer um gato em uma foto, a IA vê milhões de fotos de gatos diferentes até aprender a identificar as características que fazem um gato ser um gato: bigodes, orelhas pontudas, rabo peludo... 🐱
> 
> Depois de aprender, a IA pode usar esse conhecimento para identificar gatos em fotos que ela nunca viu antes! Incrível, né? 🤩
> 
> **E onde mais podemos encontrar a IA?** 😮
> 
> A IA está sendo usada para criar robôs que ajudam as pessoas em casa, carros que dirigem sozinhos e até mesmo para ajudar os médicos a encontrarem curas para doenças! 😮
> 
> **Que tal criarmos nossa própria IA?** 🤯
> 
> Existem ferramentas online muito legais que permitem que você crie seus próprios projetos de IA! Você pode ensinar um computador a reconhecer objetos, criar jogos inteligentes e muito mais!
> 
> **Mas, espere! A IA pode ser usada para o mal?** 😟
> 
> Sim, Cafe, como toda ferramenta poderosa, a IA pode ser usada de forma irresponsável. Por isso, é importante que as pessoas que criam e usam a IA pensem sempre em como ela pode ajudar a todos e não prejudicar ninguém.
> 
> **E como podemos garantir que a IA seja usada para o bem?** 🤔
> 
> Precisamos aprender sobre IA, discutir seus impactos e criar regras para garantir que ela seja usada de forma ética e responsável. Assim, podemos aproveitar todos os benefícios que essa tecnologia incrível tem a oferecer! 😊
> 
> Cafe, essa foi só uma pequena introdução ao mundo da IA! Espero que você tenha se divertido e aprendido muito! 😄
> 
> E lembre-se: o futuro da IA está em nossas mãos! 😉


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


Espero que este relatório detalhado ajude você a otimizar seus estudos e aprendizado! 😊
